In [ ]:
# # Input your kaggle json file

# from google.colab import files 
# data_to_load = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
# !pip install -q kaggle
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !ls ~/.kaggle
# !chmod 600 /root/.kaggle/kaggle.json 

kaggle.json


In [ ]:
DATA_DIR='/content/vipdata' ## where the data location would be

The dataset should be in the format below: 

```
DATA_DIR
> train
--> train
----> 00001
----> 00002
      .....

> valid
--> valid
----> 00081
----> 00082
      .....

> test1
--> test1
----> 00001
----> 00002
      .....

```

In [ ]:
!mkdir vipdata
%cd vipdata
!kaggle datasets download -d awsaf49/ieee-vip-cup-2021-train-val-dataset --unzip
%cd ..

/content/vipdata
100% 3.16G/3.16G [00:53<00:00, 29.3MB/s]
100% 3.16G/3.16G [00:53<00:00, 63.1MB/s]
/content


In [ ]:
%%writefile generate_cover.py

import argparse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2,os
from glob import glob
import scipy.io as scio
import scipy
import shutil
from matplotlib.colors import rgb2hex
from skimage.morphology import reconstruction
from skimage.morphology import disk
from skimage.filters.rank import gradient
import skimage
from tqdm import tqdm
os.system('pip install git+https://github.com/albumentations-team/albumentations')
import albumentations as A

kplines = [(0, 1), (1, 2), (12, 2), (12, 3), (3, 4), (4, 5), (6, 7),
            (7, 8), (8, 12), (12, 9), (9, 10), (10, 11), (12, 13)]
name2idx = {
    "Right ankle":0,
    "Right knee":1,
    "Right hip":2,
    "Left hip":3,
    "Left knee":4,
    "Left ankle":5,
    "Right wrist":6,
    "Right elbow":7,
    "Right shoulder":8,
    "Left shoulder":9,
    "Left elbow":10,
    "Left wrist":11,
    "Thorax":12,
    "Head top":13, 
}
idx2name = {v:k for k,v in name2idx.items()}

def load_kps(kp_path):
    gt  = scio.loadmat(kp_path)['joints_gt']
    kps = gt[:2].transpose(2, 1, 0) # => (num_image, num_limb, 2) => (None, 14, 2)
    return kps.astype(int)

def draw_kp(img, kps, kplines, line_th=1, circle_th=2, fontScale=1, text_th =2, text=False):
    """
    Args:
        img      : image (R, G, B)
        kps      : keypoints (num_points, 2)
        kplines  : limb line tuple index
        text     : show text or not
    Returns:
        drew image
    """
    cmap   = plt.get_cmap('rainbow')
    colors = [cmap(i) for i in np.linspace(0, 1, len(kps) + 2)]
    colors = [(c[2] * 255, c[1] * 255, c[0] * 255) for c in colors]
    for idx, kpline in enumerate(kplines):
        img = cv2.line(img.astype(float), tuple(kps[kpline[0]]), tuple(kps[kpline[1]]), thickness=line_th,
                       color=colors[idx], lineType=cv2.LINE_AA)
    for idx in range(len(kps)):
        color = colors[idx]
        img = cv2.circle(img.astype(float),tuple(kps[idx]), circle_th, color , cv2.FILLED)
        if text:
            w = img.shape[1]
            px = kps[idx][0]
            py = kps[idx][1]
            if px>w//2:
                px+=10
                color = (255,0,0)
            else:
                px-=30
                color = (0,0,255)
            img = cv2.putText(img, str(idx), (px, py), cv2.FONT_HERSHEY_SIMPLEX, 
                               fontScale=fontScale, color=color, thickness=text_th, lineType=cv2.LINE_AA)
    return img.astype('uint8')


            
def apply_selective_thorax(main_img,transformed_image,point):
    extra=int(point)
    new_img=main_img.copy()
    new_img[extra:,:]=transformed_image[extra:,:]
    return new_img

def pad_across_width(image):
    shapes=image.shape
    h=shapes[0]
    w=shapes[1]
    if len(shapes)>2:
        c=shapes[-1]

    diff=h-w
    side1=int(diff/2)
    side2=diff-side1

    s1=np.zeros((h,side1)).astype(image.dtype) if len(shapes)==2 else np.zeros((h,side1,c)).astype(image.dtype) 
    s2=np.zeros((h,side2)).astype(image.dtype) if len(shapes)==2 else np.zeros((h,side2,c)).astype(image.dtype) 

    new_image=image.copy()
    new_image=np.concatenate([s1,new_image,s2],axis=1)
    return new_image
tpoint=name2idx["Thorax"]

def cover_gen(points,image,return_coord=False):
    cover=np.zeros(image.shape,dtype=image.dtype)
    
    cover_corner_right=points[:,0].max()+20
    cover_corner_left=points[:,0].min()-20
    low=150
    hthorax=points[tpoint][1]

    if len(image.shape)>2:
        cover[hthorax:low,cover_corner_left:cover_corner_right,0]=1# if image.dtype=='uint8' else 255.0
        cover=cover[:,:,0]
    else:
        cover[hthorax:low,cover_corner_left:cover_corner_right]=1 #if image.dtype=='uint8' else 255.0
    
    if return_coord:
        return ((cover_corner_left,hthorax),(cover_corner_right,low)),cover
    return cover

def combine(image,path1,path2):
    albu=A.Compose([
                   A.augmentations.domain_adaptation.HistogramMatching(path2,blend_ratio=(0.5, 0.9),p=1),
    A.augmentations.domain_adaptation.FDA(path1,p=1,beta_limit=0.05)])
    albu_im= albu(image=image)['image']
    return albu_im

def combiner(image1,image2,cover,final_channel=1):
    if len(cover.shape)==2:
        cover=np.expand_dims(cover,axis=-1)
    if len(image2.shape)==2:
        image2=np.expand_dims(image2,axis=-1)
    
    if len(image1.shape)>2&image1.shape[-1]!=1:
        image1=np.expand_dims(image1[:,:,0],axis=-1)
    
    total=(1-cover)*image1+cover*image2
    if final_channel==3:
        return np.stack([total,total,total])
    return total

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--data-dir', type=str, default='/kaggle/input/ieee-vip-cup-2021-train-val-dataset/', help='main directory of data')
    parser.add_argument('--save-dir', type=str, default='/kaggle/working/VIP',help="where to save files, a new directory")
    opt = parser.parse_args()
    
    save_at=opt.save_dir#'/kaggle/working'
    global_path=opt.data_dir
    
#     if os.path.exists(save_at):
#         save_at=os.path.join(save_at,'VIP')
        
    print('Copying ...')
    try:
        shutil.copytree(global_path,save_at)
    except:
        shutil.rmtree(save_at)
        shutil.copytree(global_path,save_at)
    
    print('Finding Train statistics...')
    
    uncover=[]
    cover1=[]
    cover2=[]
    files_dir=os.path.join(global_path,'train')
    if len(os.listdir(files_dir))<4:
        files_dir=os.path.join(files_dir,'train')
    files=os.listdir(files_dir)
    uncover_images_list=[]
    for f in files:
        current_dir=os.path.join(files_dir,f,'IR')
        if 'uncover' in os.listdir(current_dir):
            uncover.append(f)
            uncover_images_list.extend(glob(os.path.join(current_dir,'uncover/*')))

        if 'cover1' in os.listdir(current_dir):
            cover1.append(f)
        if 'cover2' in os.listdir(current_dir):
            cover2.append(f)

    print(f'Uncover dirs : {len(uncover)}, Total files: {len(uncover_images_list)}')
    print(f'Cover1 dirs : {len(cover1)}')
    print(f'Cover2 dirs : {len(cover2)}')
    
    cover2_path=['image_000027.png',
     'image_000041.png',
     'image_000038.png',
     'image_000023.png']
    
    cover2_path=[os.path.join(files_dir,'00062/IR/cover2',i) for i in cover2_path]
    print('Selected cover images are : ',cover2_path)
    
    copy_dir=os.path.join(save_at,'train')
    if len(os.listdir(copy_dir))<5:
        copy_dir=os.path.join(copy_dir,'train')
        
    print('Generating cover images from uncover')   
    for file_num in tqdm(sorted(uncover)):
        image_dir=sorted(glob(os.path.join(copy_dir,file_num,'IR','uncover','*')))
        keypoints_path=os.path.join(copy_dir,file_num,'joints_gt_IR.mat')

        kp=load_kps(keypoints_path)
        for i in range(len(image_dir)):
            image_path=image_dir[i]

            image=cv2.imread(image_path)
            point=kp[i-1]
            ((x1,y1),(x2,y2)),cover=cover_gen(point,image[:,:,0],return_coord=True)
            final_im=combine(image,cover2_path,cover2_path)
            final_im=combiner(image,final_im,cover)

            cv2.imwrite(image_dir[i],final_im)
    print('Completed')

Writing generate_cover.py


In [ ]:
SAVE_DIR='/content/VIP'

In [ ]:
!python generate_cover.py --data-dir $DATA_DIR --save-dir $SAVE_DIR

  Cloning https://github.com/albumentations-team/albumentations to /tmp/pip-req-build-uth5rwu4
  Running command git clone -q https://github.com/albumentations-team/albumentations /tmp/pip-req-build-uth5rwu4
  Created wheel for albumentations: filename=albumentations-1.0.3-py3-none-any.whl size=98951 sha256=cf311f7b401af660691beb406699d8f961dcc2325eee2de67916cce22b170211
  Stored in directory: /tmp/pip-ephem-wheel-cache-5ts0rqu9/wheels/3a/25/ed/ec3b518e7a332d7f0a3bb37c280e1b784cf2f79b94b3c7d00b
Successfully built albumentations
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12
Copying ...
Finding Train statistics...
Uncover dirs : 30, Total files: 1350
Cover1 dirs : 25
Cover2 dirs : 25
Selected cover images are :  ['/content/vipdata/train/train/00062/IR/cover2/image_000027.png', '/content/vipdata/train/train/00062/IR/cover2/image_000041.png', '/conten